In [16]:
%pip install -r requirements.txt -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import urllib.request
import pandas as pd
from dotenv import load_dotenv
from pinecone import Pinecone

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

from langchain_openai import ChatOpenAI  
from langchain.chains import RetrievalQA 
from google.oauth2 import service_account
from io import BytesIO
from googleapiclient.discovery import build 

In [18]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key

pinecone_api_key = os.getenv('PINECONE_API_KEY')
os.environ["PINECONE_API_KEY"] = pinecone_api_key

credentials = service_account.Credentials.from_service_account_info({
    "type": os.getenv("GOOGLE_TYPE"),
    "project_id": os.getenv("GOOGLE_PROJECT_ID"),
    "private_key_id": os.getenv("GOOGLE_PRIVATE_KEY_ID"),
    "private_key": os.getenv("GOOGLE_PRIVATE_KEY").replace("\\n", "\n"),
    "client_email": os.getenv("GOOGLE_CLIENT_EMAIL"),
    "client_id": os.getenv("GOOGLE_CLIENT_ID"),
    "auth_uri": os.getenv("GOOGLE_AUTH_URI"),
    "token_uri": os.getenv("GOOGLE_TOKEN_URI"),
    "auth_provider_x509_cert_url": os.getenv("GOOGLE_AUTH_PROVIDER_X509_CERT_URL"),
    "client_x509_cert_url": os.getenv("GOOGLE_CLIENT_X509_CERT_URL")
})

service = build('drive', 'v3', credentials=credentials)
file_id = os.getenv('FILE_ID')
request = service.files().get_media(fileId=file_id)
file_data = BytesIO(request.execute())
df = pd.read_csv(file_data, header=None, names=['text', 'Category'])

In [19]:
embeddings = OpenAIEmbeddings()

# 'text_column' の各セルをベクトル化してリストに保存
df['vectorized'] = df['text'].apply(lambda x: embeddings.embed_query(x))

# ベクトルリストをデータフレームに展開（各要素を別々のカラムに）
vectorized_df = pd.DataFrame(df['vectorized'].tolist(), index=df.index)

# 元のDataFrameとベクトル化したDataFrameを結合
output_df = pd.concat([df.drop(columns=['vectorized']), vectorized_df], axis=1)

In [20]:
pc = Pinecone(api_key=pinecone_api_key)
pinecone_index = pc.Index("sample-db")
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'sample-db-rdebc4f.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'sample-db',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [21]:
original_texts = output_df.iloc[:, 0]
original_category = output_df.iloc[:, 1]
vectorized_data_only = output_df.iloc[:, 2:]

In [22]:
for i in range(len(vectorized_data_only)):
    pinecone_index.upsert(
        vectors = [
            {
                'id': str(i+1),
                'values': vectorized_data_only.T[i],
                'metadata': {"text": original_texts[i], "Category": original_category[i]}
            }
        ]
    )

In [23]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'sample-db-rdebc4f.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'sample-db',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [24]:
env="aws"
index_name = "sample-db"


embeddings = OpenAIEmbeddings()

#vectorstore = PineconeStore.from_documents(index_name, embeddings,"text")
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings,pinecone_api_key=pinecone_api_key)
#retriever = PineconeVectorStore(search_type="similarity", search_kwargs={"k": 5})

In [25]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# プロンプトテンプレートの定義
prompt_template = """Use the following pieces of context: {context} 
感情が伝わるように「！」など感情表現豊かにしてください。全てに「！」をつけないでください、適切なタイミングで使うようにして。また一問一答のように質問にだけ応えて。: {question}"""
PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

# completion llm  
llm = ChatOpenAI(  
    model_name='gpt-4o-mini',  
    temperature=0,
)  

# RetrievalQAの設定
qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT, "document_variable_name": "context"} 
)  

# クエリの実行
q = '休みの日は何をしていますか？'
result = qa.invoke({"query": q})
display("Answer:", result['result'])


'Answer:'

'休みの日は読書をしたり、散歩を楽しんだり、プログラミングに没頭したりしています！特に自然の中を歩くのが好きです！'

In [26]:
'''
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# プロンプトテンプレートの定義
prompt_template = """Use the following pieces of context: {context} 
感情が伝わるように「！」など感情表現豊かにしてください。全てに「！」をつけないでください、適切なタイミングで使うようにして。また一問一答のように質問にだけ応えて。: {question}"""
PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

# completion llm  
llm = ChatOpenAI(  
    model_name='gpt-4o-mini',  
    temperature=0,
)  

# RetrievalQAの設定
qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT, "document_variable_name": "context"} 
)  

# インタラクティブな会話ループ
context = ""  # 初期文脈
while True:
    q = input("あなたの質問: ")  # ユーザーからの質問を取得
    result = qa.invoke({"query": q, "context": context})  # 質問を実行
    print("回答:", result['result'])  # モデルの応答を表示
    
    # 次の質問のために文脈を更新
    context += f"\n質問: {q}\n回答: {result['result']}"
    
    # 終了条���
    if q.lower() in ["exit", "終了", "やめる"]:
        break
'''

'\nfrom langchain.prompts import PromptTemplate\nfrom langchain_openai import ChatOpenAI\nfrom langchain.chains import RetrievalQA\n\n# プロンプトテンプレートの定義\nprompt_template = """Use the following pieces of context: {context} \n感情が伝わるように「！」など感情表現豊かにしてください。全てに「！」をつけないでください、適切なタイミングで使うようにして。また一問一答のように質問にだけ応えて。: {question}"""\nPROMPT = PromptTemplate(\n    template=prompt_template, \n    input_variables=["context", "question"]\n)\n\n# completion llm  \nllm = ChatOpenAI(  \n    model_name=\'gpt-4o-mini\',  \n    temperature=0,\n)  \n\n# RetrievalQAの設定\nqa = RetrievalQA.from_chain_type(  \n    llm=llm,  \n    chain_type="stuff",  \n    retriever=vectorstore.as_retriever(),\n    chain_type_kwargs={"prompt": PROMPT, "document_variable_name": "context"} \n)  \n\n# インタラクティブな会話ループ\ncontext = ""  # 初期文脈\nwhile True:\n    q = input("あなたの質問: ")  # ユーザーからの質問を取得\n    result = qa.invoke({"query": q, "context": context})  # 質問を実行\n    print("回答:", result[\'result\'])  # モデルの応答を表示\n    \n    # 次の質問のために文脈を更新\n

In [27]:
q='彼女はいますか'
result = qa.invoke({"query": q})
display("Answer:", result['result'])

'Answer:'

'いいえ、彼女はいません！今は勉強に集中しています！'

In [30]:
q='大学と学部と学科を教えてください'
result = qa.invoke({"query": q})
display("Answer:", result['result'])

'Answer:'

'僕の大学は東京国際工科専門職大学です！学部は工科学部で、学科は情報工学科です！'

In [32]:
q='研究室のリンクとかありますか？'
result = qa.invoke({"query": q})
display("Answer:", result['result'])

'Answer:'

'はい！研究室のリンクはこちらです！[塩尻（斎藤）亜希 研究室](https://shiojirilab.wixsite.com/shiojirilab/%E3%82%A2%E3%82%AF%E3%82%BB%E3%82%B9)です！ぜひ訪れてみてください！'